In [1]:
# 1. Read API from text file
# f = open("api.txt", "r")
# api_key = f.read()
api_key = '4f8acc9d2aed40##^$$&' # get API key from: https://app.assemblyai.com/

print('1. API is read ...: ', api_key)

1. API is read ...:  4f8acc9d2aed40##^$$&


In [ ]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 3.6 MB/s 


In [ ]:
# 2. Retrieving audio file from YouTube video
from pytube import YouTube
video = YouTube("https://www.youtube.com/watch?v=mkVjrB8g6mM")

yt = video.streams.get_audio_only()

yt.download()

'/content/What is the best way to learn data science.mp4'

In [ ]:
import os
current_dir = os.getcwd()

for file in os.listdir(current_dir):
    if file.endswith(".mp4"):
        mp4_file = os.path.join(current_dir, file)
        print(mp4_file)

print('2. Audio file has been retrieved from YouTube video: ', mp4_file)

/content/What is the best way to learn data science.mp4
2. Audio file has been retrieved from YouTube video:  /content/What is the best way to learn data science.mp4


In [ ]:
# 3. Upload YouTube audio file to AssemblyAI

import sys
import time
import requests

filename = mp4_file

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data
 
headers = {'authorization': api_key}
response = requests.post('https://api.assemblyai.com/v2/upload',
                         headers=headers,
                         data=read_file(filename))

audio_url = response.json()['upload_url']

print('3. YouTube audio file has been uploaded to AssemblyAI: ', audio_url)

3. YouTube audio file has been uploaded to AssemblyAI:  https://cdn.assemblyai.com/upload/181169c7-0b22-4ced-9750-58074934d984


In [ ]:
# 4. Transcribe uploaded audio file

import requests

endpoint = "https://api.assemblyai.com/v2/transcript"

json = {
  "audio_url": audio_url
}

headers = {
    "authorization": api_key,
    "content-type": "application/json"
}

transcript_input_response = requests.post(endpoint, json=json, headers=headers)

print('4. Transcribing uploaded file: ', transcript_input_response.json())

4. Transcribing uploaded file:  {'id': 'oqnbizf6zq-ea7e-4844-85a3-44c5657559e6', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://cdn.assemblyai.com/upload/181169c7-0b22-4ced-9750-58074934d984', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': False, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'content_safety': False, 'iab_categories': False, 'content_safety_labels': {}, 'iab_categories_result': {}, 'language_detection': False, 'custom_sp

In [ ]:
# 5. Extract transcript ID

transcript_id = transcript_input_response.json()["id"]

print('5. Extract transcript ID: ',transcript_id)

5. Extract transcript ID:  oqnbizf6zq-ea7e-4844-85a3-44c5657559e6


In [ ]:
# 6. Retrieve transcription results
endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"
headers = {
    "authorization": api_key,
}

transcript_output_response = requests.get(endpoint, headers=headers)

print('6. Retrieve transcription results: ',transcript_output_response)

6. Retrieve transcription results:  <Response [200]>


In [ ]:
transcript_output_response.json()['status']

'queued'

In [ ]:
from time import sleep

while transcript_output_response.json()['status'] != 'completed':
  sleep(5)
  print('Transcription is processing ...')
  transcript_output_response = requests.get(endpoint, headers=headers)


print(transcript_output_response.json()["status"])

Transcription is processing ...
Transcription is processing ...
Transcription is processing ...
completed


In [ ]:
transcript_output_response.json()

{'acoustic_model': 'assemblyai_default',
 'audio_duration': 15,
 'audio_end_at': None,
 'audio_start_from': None,
 'audio_url': 'https://cdn.assemblyai.com/upload/181169c7-0b22-4ced-9750-58074934d984',
 'auto_chapters': False,
 'auto_highlights': False,
 'auto_highlights_result': None,
 'boost_param': None,
 'chapters': None,
 'confidence': 0.90008025,
 'content_safety': False,
 'content_safety_labels': {'results': [],
  'status': 'unavailable',
  'summary': {}},
 'custom_spelling': None,
 'disfluencies': False,
 'dual_channel': None,
 'entities': None,
 'entity_detection': False,
 'filter_profanity': False,
 'format_text': True,
 'iab_categories': False,
 'iab_categories_result': {'results': [],
  'status': 'unavailable',
  'summary': {}},
 'id': 'oqnbizf6zq-ea7e-4844-85a3-44c5657559e6',
 'language_code': 'en_us',
 'language_detection': False,
 'language_model': 'assemblyai_default',
 'punctuate': True,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': Non

In [ ]:
# 7. Print transcribed text
print('Output:\n')
print(transcript_output_response.json()["text"])

Output:

So what's the best way to learn data science? Well, the best way to learn data science is to do data science. If you're into this type of content, you want to consider subscribing for more tutorials and concept. Video.


In [ ]:
# 8. Save transcribed text to file

# Save as TXT file
yt_txt = open('yt.txt', 'w')
yt_txt.write(transcript_output_response.json()["text"])
yt_txt.close()

In [ ]:
# Save as SRT file
srt_endpoint = endpoint + "/srt"
srt_response = requests.get(srt_endpoint, headers=headers)

with open("yt.srt", "w") as _file:
    _file.write(srt_response.text)